In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils
import torch.utils.data
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from math import pi
import pickle
import time

In [2]:
sigma = torch.tensor(0.05)

In [3]:
device = torch.device('cpu')

# Define the exact solution
def exact_solution(x, t):
    return torch.sin(x)*torch.cos(pi*t)

def initial_condition(x):
    return torch.sin(x)*(1 + sigma*torch.randn(x.shape)).to(device)

def initial_condition_t(x):
    return 0*torch.cos(x)


In [4]:
# assigning number of points
initial_pts = 500
left_boundary_pts = 500
right_boundary_pts = 500
residual_pts = 10000

# Type of optimizer (ADAM or LBFGS)
opt_type = "LBFGS"

x_init = 8*pi*torch.rand((initial_pts,1)) # initial pts
t_init = 0*x_init
init = torch.cat([x_init, t_init],1).to(device)
u_init = initial_condition(init[:,0]).reshape(-1, 1)
u_init_t = 0*initial_condition(init[:,0]).reshape(-1, 1)

xb_left = torch.zeros((left_boundary_pts, 1)) # left spatial boundary
tb_left = torch.rand((left_boundary_pts, 1)) #
b_left = torch.cat([xb_left, tb_left ],1).to(device)
u_b_l = 0*torch.sin(tb_left)

xb_right = 8*pi*torch.ones((right_boundary_pts, 1)) # right spatial boundary
tb_right = torch.rand((right_boundary_pts, 1)) # right boundary pts
b_right = torch.cat([xb_right, tb_right ],1).to(device)
u_b_r = 0*torch.sin(2*pi - tb_right)

x_interior = 8*pi*torch.rand((residual_pts, 1))
t_interior = torch.rand((residual_pts, 1))
interior = torch.cat([x_interior, t_interior],1).to(device)

training_set = DataLoader(torch.utils.data.TensorDataset(init.to(device), u_init.to(device), u_init_t.to(device), b_left.to(device),  b_right.to(device), u_b_l.to(device), u_b_r.to(device)), batch_size=500, shuffle=False)


In [5]:
class NeuralNet(nn.Module):

    def __init__(self, input_dimension, output_dimension, n_hidden_layers, neurons):
        super(NeuralNet, self).__init__()
        # Number of input dimensions n
        self.input_dimension = input_dimension
        # Number of output dimensions m
        self.output_dimension = output_dimension
        # Number of neurons per layer
        self.neurons = neurons
        # Number of hidden layers
        self.n_hidden_layers = n_hidden_layers
        # Activation function
        self.activation = nn.Tanh()

        self.input_layer = nn.Linear(self.input_dimension, self.neurons)
        self.hidden_layers = nn.ModuleList([nn.Linear(self.neurons, self.neurons) for _ in range(n_hidden_layers)])
        self.output_layer = nn.Linear(self.neurons, self.output_dimension)

    def forward(self, x):
        # The forward function performs the set of affine and non-linear transformations defining the network
        # (see equation above)
        x = self.activation(self.input_layer(x))
        for k, l in enumerate(self.hidden_layers):
            x = self.activation(l(x))
        return self.output_layer(x)



In [6]:
# Model definition
my_network = NeuralNet(input_dimension = init.shape[1], output_dimension = u_init.shape[1], n_hidden_layers=4, neurons=200)
model_state_dict = torch.load('PINN_EB.pth', map_location=torch.device('cpu'))
my_network = my_network.to(device)

# def init_xavier(model, retrain_seed):
#     torch.manual_seed(retrain_seed)
#     def init_weights(m):
#         if type(m) == nn.Linear and m.weight.requires_grad and m.bias.requires_grad:
#             g = nn.init.calculate_gain('tanh')
#             torch.nn.init.xavier_uniform_(m.weight, gain=g)
#             #torch.nn.init.xavier_normal_(m.weight, gain=g)
#             m.bias.data.fill_(0)
#     model.apply(init_weights)

# # Random Seed for weight initialization
# retrain = 128
# # Xavier weight initialization
# init_xavier(my_network, retrain)

if opt_type == "ADAM":
    optimizer_ = optim.Adam(my_network.parameters(), lr=0.001)
elif opt_type == "LBFGS":
    optimizer_ = optim.LBFGS(my_network.parameters(), lr=0.1, max_iter=1, max_eval=50000, tolerance_change=1.0 * np.finfo(float).eps)
else:
    raise ValueError("Optimizer not recognized")


def fit(model, training_set, interior, num_epochs, optimizer, p, verbose=True):
    history = list()

    # Loop over epochs
    for epoch in range(num_epochs):
        if verbose: print("################################ ", epoch, " ################################")

        running_loss = list([0])

        # Loop over batches
        for j, (initial, u_initial, u_initial_t, bd_left, bd_right, ubl, ubr) in enumerate(training_set):
            def closure():
                # zero the parameter gradients
                optimizer.zero_grad()
                # for initial
                initial.requires_grad = True
                u_initial_pred_ = model(initial)
                inputs = torch.ones(initial_pts, 1).to(device)
                grad_u_init = torch.autograd.grad(u_initial_pred_, initial, grad_outputs=inputs, create_graph=True)[0]
                u_init_t = grad_u_init[:, 1].reshape(-1, )

                # for left boundary
                bd_left.requires_grad = True
                bd_left_pred_ = model(bd_left)
                inputs = torch.ones(left_boundary_pts, 1).to(device)
                grad_bd_left = torch.autograd.grad(bd_left_pred_, bd_left, grad_outputs=inputs, create_graph=True)[0]
                u_bd_x_left = grad_bd_left[:, 0].reshape(-1, )
                inputs = torch.ones(left_boundary_pts, 1).reshape(-1, ).to(device)
                grad_u_bd_x_left = torch.autograd.grad(u_bd_x_left, bd_left, grad_outputs=inputs, create_graph=True)[0]
                u_bd_xx_left = grad_u_bd_x_left[:, 0].reshape(-1, )
                inputs = torch.ones(left_boundary_pts, 1).reshape(-1, ).to(device)
                grad_u_bd_xx_left = torch.autograd.grad(u_bd_xx_left, bd_left, grad_outputs=inputs, create_graph=True)[0]
                u_bd_xxx_left = grad_u_bd_xx_left[:, 0].reshape(-1, )

                # for right boundary
                bd_right.requires_grad = True
                bd_right_pred_ = model(bd_right)
                inputs = torch.ones(right_boundary_pts, 1).to(device)
                grad_bd_right = torch.autograd.grad(bd_right_pred_, bd_right, grad_outputs=inputs, create_graph=True)[0]
                u_bd_x_right = grad_bd_right[:, 0].reshape(-1, )
                inputs = torch.ones(right_boundary_pts, 1).reshape(-1, ).to(device)
                grad_u_bd_x_right = torch.autograd.grad(u_bd_x_right, bd_right, grad_outputs=inputs, create_graph=True)[0]
                u_bd_xx_right = grad_u_bd_x_right[:, 0].reshape(-1, )

                # residual calculation
                interior.requires_grad = True
                u_hat = model(interior)
                inputs = torch.ones(residual_pts, 1).to(device)
                grad_u_hat = torch.autograd.grad(u_hat, interior, grad_outputs=inputs, create_graph=True)[0]

                u_x = grad_u_hat[:, 0].reshape(-1, )
                inputs = torch.ones(residual_pts, 1).reshape(-1, ).to(device)
                grad_u_x = torch.autograd.grad(u_x, interior, grad_outputs=inputs, create_graph=True)[0]
                u_xx = grad_u_x[:, 0].reshape(-1, )
                inputs = torch.ones(residual_pts, 1).reshape(-1, ).to(device)
                grad_u_xx = torch.autograd.grad(u_xx, interior, grad_outputs=inputs, create_graph=True)[0]
                u_xxx = grad_u_xx[:, 0].reshape(-1, )
                inputs = torch.ones(residual_pts, 1).reshape(-1, ).to(device)
                grad_u_xxx = torch.autograd.grad(u_xxx, interior, grad_outputs=inputs, create_graph=True)[0]
                u_xxxx = grad_u_xxx[:, 0].reshape(-1, )

                u_t = grad_u_hat[:, 1]
                inputs = torch.ones(residual_pts, 1).reshape(-1, ).to(device)
                grad_u_t = torch.autograd.grad(u_t, interior, grad_outputs=inputs, create_graph=True)[0]
                u_tt = grad_u_t[:, 1].reshape(-1, )

                # Item 1. below

                loss_ic = torch.mean((u_initial_pred_.reshape(-1, ) - u_initial.reshape(-1, )) ** p) + \
                          torch.mean((u_init_t.reshape(-1, )) ** p)
                loss_pde = torch.mean((u_tt.reshape(-1, ) + u_xxxx.reshape(-1, ) - (2-pi**2)*torch.sin(interior[:,0])*torch.cos(pi*interior[:,1])) ** p)
                loss_left_b = torch.mean((u_bd_x_left.reshape(-1, ) - ubl.reshape(-1, )) ** p) + \
                              torch.mean((u_bd_xx_left.reshape(-1, ) - ubl.reshape(-1, )) ** p)
                loss_right_b = torch.mean((bd_right_pred_.reshape(-1, ) - ubr.reshape(-1, )) ** p) + \
                               torch.mean((u_bd_xx_right.reshape(-1, ) - ubr.reshape(-1, )) ** p)

                loss = loss_ic + loss_pde + loss_left_b + loss_right_b

                # Item 2. below
                loss.backward()
                # Compute average training loss over batches for the current epoch
                running_loss[0] += loss.item()
                return loss

            # Item 3. below
            optimizer.step(closure=closure)

        print('Loss: ', (running_loss[0] / len(training_set)))
        history.append(running_loss[0])

    return history


In [ ]:
n_epochs = 1500
start_time = time.time()
history = fit(my_network, training_set, interior, n_epochs, optimizer_, p=2, verbose=True )
end_time = time.time()
total_time = end_time - start_time
print("Training time: {:.2f} seconds".format(total_time))


with open('PINN_5_noise_tl.pkl', 'wb') as f:
    pickle.dump(history, f)

f.close()

model_state_dict = my_network.state_dict()

# Save the model state dictionary to a file
torch.save(model_state_dict, 'PINN_5_noise_tl.pth')


################################  0  ################################


/data/localhome/tkapoor/.local/lib/python3.8/site-packages/torch/autograd/__init__.py:234: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return Variable._execution_engine.run_backward(


Loss:  16.014644622802734
################################  1  ################################
Loss:  16.014142990112305
################################  2  ################################
Loss:  16.010597229003906
################################  3  ################################
Loss:  16.007429122924805
################################  4  ################################
Loss:  16.004562377929688
################################  5  ################################
Loss:  16.001949310302734
################################  6  ################################
Loss:  15.999544143676758
################################  7  ################################
Loss:  15.99730396270752
################################  8  ################################
Loss:  15.995192527770996
################################  9  ################################
Loss:  15.99317741394043
################################  10  ################################
Loss:  15.991230964660645
###############

Loss:  12.83405590057373
################################  86  ################################
Loss:  12.794971466064453
################################  87  ################################
Loss:  12.757987976074219
################################  88  ################################
Loss:  12.720791816711426
################################  89  ################################
Loss:  12.684744834899902
################################  90  ################################
Loss:  12.650808334350586
################################  91  ################################
Loss:  12.617951393127441
################################  92  ################################
Loss:  12.584494590759277
################################  93  ################################
Loss:  12.549375534057617
################################  94  ################################
Loss:  12.511905670166016
################################  95  ################################
Loss:  12.4693021774292
#######

Loss:  9.171722412109375
################################  171  ################################
Loss:  9.13016414642334
################################  172  ################################
Loss:  9.09145736694336
################################  173  ################################
Loss:  9.054656982421875
################################  174  ################################
Loss:  9.021906852722168
################################  175  ################################
Loss:  8.990970611572266
################################  176  ################################
Loss:  8.962967872619629
################################  177  ################################
Loss:  8.933927536010742
################################  178  ################################
Loss:  8.906444549560547
################################  179  ################################
Loss:  8.878899574279785
################################  180  ################################
Loss:  8.848917007446289
#######

Loss:  7.088838577270508
################################  256  ################################
Loss:  7.07680082321167
################################  257  ################################
Loss:  7.066713809967041
################################  258  ################################
Loss:  7.05665922164917
################################  259  ################################
Loss:  7.04733419418335
################################  260  ################################
Loss:  7.038116455078125
################################  261  ################################
Loss:  7.029101848602295
################################  262  ################################
Loss:  7.020209312438965
################################  263  ################################
Loss:  7.011499881744385
################################  264  ################################
Loss:  7.002997398376465
################################  265  ################################
Loss:  6.994593620300293
########

Loss:  6.317900657653809
################################  341  ################################
Loss:  6.3031182289123535
################################  342  ################################
Loss:  6.287880897521973
################################  343  ################################
Loss:  6.273482799530029
################################  344  ################################
Loss:  6.259820461273193
################################  345  ################################
Loss:  6.246296405792236
################################  346  ################################
Loss:  6.232451915740967
################################  347  ################################
Loss:  6.219444274902344
################################  348  ################################
Loss:  6.206212043762207
################################  349  ################################
Loss:  6.194238185882568
################################  350  ################################
Loss:  6.182246208190918
####

Loss:  5.439902305603027
################################  426  ################################
Loss:  5.430243492126465
################################  427  ################################
Loss:  5.420330047607422
################################  428  ################################
Loss:  5.409718990325928
################################  429  ################################
Loss:  5.399316787719727
################################  430  ################################
Loss:  5.388945579528809
################################  431  ################################
Loss:  5.377865791320801
################################  432  ################################
Loss:  5.367147922515869
################################  433  ################################
Loss:  5.356409072875977
################################  434  ################################
Loss:  5.345238208770752
################################  435  ################################
Loss:  5.334343433380127
#####

In [ ]:
# # loading model

# # Load the history from the pickle file
# with open('PINN_EB.pkl', 'rb') as f:
#     history = pickle.load(f)

# # # Load the model architecture
# # my_network = your_model_module.YourModelClass()  # Instantiate your model class

# # Load the saved model state dictionary
# model_state_dict = torch.load('PINN_EB.pth',  map_location=torch.device('cpu'))

# # Load the model weights
# my_network.load_state_dict(model_state_dict)


In [ ]:
x_test = torch.linspace(0, 8*pi, 10000).reshape(-1,1)
t_test = torch.ones((10000,1))
test = torch.cat([x_test, t_test],1)
u_test = exact_solution(x_test, t_test).reshape(-1,1)
my_network = my_network.cpu()
u_test_pred = my_network(test).reshape(-1,1)

# Compute the relative L2 error norm (generalization error)
relative_error_test = torch.mean((u_test_pred - u_test)**2)/torch.mean(u_test**2)
print("Relative Error Test: ", relative_error_test.detach().numpy()*100, "%")


In [ ]:
x_test = 8*pi*torch.rand(100000).reshape(-1,1)
t_test = torch.rand(100000).reshape(-1,1)
test = torch.cat([x_test, t_test],1)
u_test = exact_solution(x_test,t_test).reshape(-1,1)
u_test_pred = my_network(test).reshape(-1,1)
relative_error = torch.abs(u_test_pred - u_test)
u_test = u_test.reshape(-1,)

# reshaping and detach numpy
x_test = x_test.reshape(-1, )
t_test = t_test.reshape(-1, )
relative_error = relative_error.reshape(-1,)
u_test_pred = u_test_pred.reshape(-1, )

x_test = x_test.detach().numpy()
t_test = t_test.detach().numpy()
u_test_pred = u_test_pred.detach().numpy()
relative_error = relative_error.detach().numpy()





plt.rcParams['figure.figsize'] = [10, 4]

from matplotlib.font_manager import FontProperties
plt.rcParams['axes.unicode_minus'] = False
font_path = 'times-new-roman.ttf'

custom_font = FontProperties(fname=font_path)


        
CS1 = plt.tricontourf(x_test, t_test, u_test_pred, 20, cmap='rainbow')
#CS1 = plt.tricontourf(x_test, t_test, u_test, 20, cmap='rainbow')
#CS1 = plt.tricontourf(x_test, t_test, relative_error, 20, cmap='rainbow')



cbar1 = plt.colorbar(CS1)
for t in cbar1.ax.get_yticklabels():
    t.set_fontproperties(custom_font)
    t.set_fontsize(20)



plt.xlabel('x', fontsize=20, fontproperties=custom_font)
plt.ylabel('t', fontsize=20, fontproperties=custom_font)
plt.xticks(fontsize=20, fontproperties=custom_font)
plt.yticks(fontsize=20, fontproperties=custom_font)
#plt.savefig('PINN_EB.pdf', dpi = 300, bbox_inches = "tight")
#plt.savefig('Exact_EB.pdf', dpi = 300, bbox_inches = "tight")
#plt.savefig('Absolute_error_EB.pdf', dpi = 300, bbox_inches = "tight")

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

# Assuming you have imported your data and defined necessary functions

# Rest of your code...

# # Convert the font size to points
font_size = 20
# ticks_font = FontProperties(family='Times New Roman', style='normal', size=font_size)





x_test = 8*pi*torch.rand(100000).reshape(-1,1)
t_test = torch.rand(100000).reshape(-1,1)
test = torch.cat([x_test, t_test],1)
u_test = exact_solution(x_test,t_test).reshape(-1,1)
u_test_pred = my_network(test).reshape(-1,1)
relative_error = torch.abs(u_test_pred - u_test)
u_test = u_test.reshape(-1,)

# reshaping and detach numpy
x_test = x_test.reshape(-1, )
t_test = t_test.reshape(-1, )
relative_error = relative_error.reshape(-1,)
u_test_pred = u_test_pred.reshape(-1, )

x_test = x_test.detach().numpy()
t_test = t_test.detach().numpy()
u_test_pred = u_test_pred.detach().numpy()
relative_error = relative_error.detach().numpy()





plt.rcParams['figure.figsize'] = [10, 4]

from matplotlib.font_manager import FontProperties
plt.rcParams['axes.unicode_minus'] = False
font_path = 'times-new-roman.ttf'

ticks_font = FontProperties(fname=font_path)

# Define the levels for contouring
levels = np.linspace(-1.5, 1.5, 20)

        
CS1 = plt.tricontourf(x_test, t_test, u_test_pred, levels, cmap='twilight')
#CS1 = plt.tricontourf(x_test, t_test, u_test, 20, cmap='twilight')
#CS1 = plt.tricontourf(x_test, t_test, relative_error, 20, cmap='rainbow')



#cbar1 = plt.colorbar(CS1)
for t in cbar1.ax.get_yticklabels():
    t.set_fontproperties(custom_font)
    t.set_fontsize(20)



plt.xlabel('x', fontsize=20, fontproperties=custom_font)
plt.ylabel('t', fontsize=20, fontproperties=custom_font)
plt.xticks(fontsize=20, fontproperties=custom_font)
plt.yticks(fontsize=20, fontproperties=custom_font)
#plt.savefig('Causal_EB.pdf', dpi = 300, bbox_inches = "tight")
#plt.savefig('PINN_EB.pdf', dpi=500, bbox_inches="tight", format='pdf', backend='cairo')
#plt.savefig('Absolute_error_EB.pdf', dpi = 300, bbox_inches = "tight")


